In [1]:
import matplotlib.pyplot as plt

In [2]:
from experiment.scenario import *
from experiment_setup.exp_utrecht_10_492_594_setup import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Setup

Check scenario settings:

In [3]:
# for i, ss in enumerate(scenario_settings):
#     print(str(i) + ": " + str(ss))

Set up experiment:

In [4]:
load_existing_experiment = True

if not load_existing_experiment:
    e = Experiment(experiment_name, experiment_dirpath, default_config, scenario_settings, scene_parts)

    print("\nSetting up experiment:")
    e.setup()
else:
    print("\nLoading existing experiment:")
    e = Experiment.load(experiment_dirpath / experiment_name, load_scenarios=True)


Loading existing experiment:
Loading experiment configuration ...
Initializing experiment ...
Loading scenarios ...


# Recon optim test

## Run crashed scenarios

So far: 24, 60

Does it crash in every 15$^{th}$ scenario?

- Scenario 55 was run independently.
- Starting with scenario 44 (sequence 1), it crashed on scenario 24 (sequence 15).
    - It crashed in optimization scenario optim_0042.
- I ran scenario 24 independently. (The rest of it.)
- Starting with scenario 13 (sequence 16), it crashed on scenario 60 (sequence 30).
    - It crashed in optimization scenario optim_0043.
- I ran scenario 60 independently. (The rest of it.)

In [5]:
from experiment.utils import get_processing_sequence

In [6]:
_, _, sequence = get_processing_sequence()

In [8]:
print(sequence) # sequence[:15], sequence[15], sequence[15:30], sequence[30], sequence[30:45], sequence[45], sequence[45:60], sequence[46:60], sequence[80:81]

[55, 44, 33, 22, 11, 0, 56, 45, 34, 23, 12, 1, 57, 46, 35, 24, 13, 2, 58, 47, 36, 25, 14, 3, 59, 48, 37, 26, 15, 4, 60, 49, 38, 27, 16, 5, 61, 50, 39, 28, 17, 6, 62, 51, 40, 29, 18, 7, 63, 52, 41, 30, 19, 8, 64, 53, 42, 31, 20, 9, 65, 54, 43, 32, 21, 10, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142]


In [11]:
sequence.index(80)

80

In [3]:
load_scenario_optimizer_states[81]

[80, 70]

In [14]:
geoflow_default_params_for_optim = {
    k: glb.geoflow_parameters_default[k]
    for k in [k2.split("_", 1)[1] for k2 in glb.geoflow_optim_parameter_space_narrow_2.keys()]
}

In [15]:
e.optimize_reconstruction_params(
    sequence=[60],
    init_points=5,
    n_iter=45,
    probe_parameter_sets=[geoflow_default_params_for_optim],
    continue_from_previous_run=True
)


Sequential optimization of reconstruction parameters.
- Sequence: 60

Optimization of reconstruction parameters for scenario `scenario_060`.

Setting up reconstruction optimization ...
- Setting up experiment ...
- Setting up optimizer ...
- Finding optimization logs of scenario: scenario_059
- scenario_059: optimization_240923-132401.log

Loading optimizer state from log file optimization_240923-132401.log ...
- Optimizer is now aware of 51 more observations, 51 total.
- Of these, 0 are actual target values.

Preparing reconstruction optimization ...
Saving experiment configuration ...
Saving scenario configurations ...
- Setting up logger ...

Loading optimization log from previous run: C:\Users\Florian\Data\city-to-scan-to-city\utrecht_10-492-594\06_reconstruction_optimization\scenario_060\optimization_240923-144322.log

Loading optimizer state from log file optimization_240923-144322.log ...
Data point [ 0.4   7.    0.2  15.    0.25  0.5   1.5 ] is not unique. 1 duplicates registe

## Rename optim test experiments

Did this for 0, 56, and 58. Not for 55, because it serves as seed for the optimization cascade.

In [21]:
e[56].setup_reconstruction_optimization()


Setting up reconstruction optimization ...
- Setting up experiment ...
- Setting up optimizer ...


In [22]:
e[56].recon_optim.load_optim_experiment()

Loading experiment configuration ...
File `scenario_settings.json` not found.
Initializing experiment ...
Loading scenarios ...


In [23]:
e[56].recon_optim.optim_experiment.rename("scenario_056_test_optim")

Renaming experiment from `scenario_056` to `scenario_056_test_optim` ...
Saving experiment configuration ...
Saving scenario configurations ...


## Set up processing sequence

First, establish for each scenario from which adjacent scenarios any optimizer logs should be loaded: `load_scenario_optimizer_states`

This is done by defining

- the error cascade, which identifies for each scenario a scenario with identical density and different, but similar (lower) error, and
- the density cascade, which identifies for each scenario a scenario with identical error and different, but similar density.

For each scenario, these two (sometimes only one) numbers are put in the mentioned list (`load_scenario_optimizer_states`).

Then, a list is generated that turns this one on its head: `next_scenarios`. It identifies for each scenario which other scenario(s) rely on it (i.e., load its optimizer logs) and can therefore be executed next after this one is done.

Of course, this does not take into account yet that many scenarios depend on two other scenarios being finished before they can be run.

This is considered when generating the `sequence` list, which identifies one possible sequence in which the scenarios can be run such that, by the time one scenario is executed, all scenarios it depends on are already finished.

In [3]:
n_scenarios = 143
seed_scenario = 55

# The error cascade is simple: Each scenario depends on the previous one in terms of numbering (and, in terms of error, the next-lower one with identical density), except those that are multiples of 11, because they already have zero error, so they cannot rely on a scenario with lower error.
# Note that a number -1 indicates that no adjacent scenario exists / should be relied on.
error_cascade = [i-1 if i not in [j * 11 for j in range(13)] else -1 for i in range(n_scenarios)]

# The density cascade is a bit more tricky: Because the seed scenario is 55, scenarios with lower density are supposed to load optimizer logs from those with the next-higher density and identical error, while scenarios with higher density than 55 (i.e., starting at 66) are supposed to load optimizer logs from those with the next-lower density.
# Note that a number -1 indicates that no adjacent scenario exists / should be relied on.
density_cascade = []

for i in range(n_scenarios):
    if i < seed_scenario:
        density_cascade.append(i+11)
    elif seed_scenario <= i < seed_scenario + 11:
        density_cascade.append(-1)
    elif i >= seed_scenario + 11:
        density_cascade.append(i-11)
    else:
        print("Crap. This shouldn't have happened.")

for i, (e, d) in enumerate(zip(error_cascade, density_cascade)):
    print(f"{i}: {e}, {d}")

0: -1, 11
1: 0, 12
2: 1, 13
3: 2, 14
4: 3, 15
5: 4, 16
6: 5, 17
7: 6, 18
8: 7, 19
9: 8, 20
10: 9, 21
11: -1, 22
12: 11, 23
13: 12, 24
14: 13, 25
15: 14, 26
16: 15, 27
17: 16, 28
18: 17, 29
19: 18, 30
20: 19, 31
21: 20, 32
22: -1, 33
23: 22, 34
24: 23, 35
25: 24, 36
26: 25, 37
27: 26, 38
28: 27, 39
29: 28, 40
30: 29, 41
31: 30, 42
32: 31, 43
33: -1, 44
34: 33, 45
35: 34, 46
36: 35, 47
37: 36, 48
38: 37, 49
39: 38, 50
40: 39, 51
41: 40, 52
42: 41, 53
43: 42, 54
44: -1, 55
45: 44, 56
46: 45, 57
47: 46, 58
48: 47, 59
49: 48, 60
50: 49, 61
51: 50, 62
52: 51, 63
53: 52, 64
54: 53, 65
55: -1, -1
56: 55, -1
57: 56, -1
58: 57, -1
59: 58, -1
60: 59, -1
61: 60, -1
62: 61, -1
63: 62, -1
64: 63, -1
65: 64, -1
66: -1, 55
67: 66, 56
68: 67, 57
69: 68, 58
70: 69, 59
71: 70, 60
72: 71, 61
73: 72, 62
74: 73, 63
75: 74, 64
76: 75, 65
77: -1, 66
78: 77, 67
79: 78, 68
80: 79, 69
81: 80, 70
82: 81, 71
83: 82, 72
84: 83, 73
85: 84, 74
86: 85, 75
87: 86, 76
88: -1, 77
89: 88, 78
90: 89, 79
91: 90, 80
92: 91, 

In [4]:
# The error and density cascades are merged into a single list, which identifies for each scenario up to two scenarios from which it should load the optimizer logs.
load_scenario_optimizer_states = []
for i, (e, d) in enumerate(zip(error_cascade, density_cascade)):
    adjacent_scenarios = []
    if e != -1:
        adjacent_scenarios.append(e)
    if d != -1:
        adjacent_scenarios.append(d)
    load_scenario_optimizer_states.append(adjacent_scenarios)

{i: l for i, l in enumerate(load_scenario_optimizer_states)}

{0: [11],
 1: [0, 12],
 2: [1, 13],
 3: [2, 14],
 4: [3, 15],
 5: [4, 16],
 6: [5, 17],
 7: [6, 18],
 8: [7, 19],
 9: [8, 20],
 10: [9, 21],
 11: [22],
 12: [11, 23],
 13: [12, 24],
 14: [13, 25],
 15: [14, 26],
 16: [15, 27],
 17: [16, 28],
 18: [17, 29],
 19: [18, 30],
 20: [19, 31],
 21: [20, 32],
 22: [33],
 23: [22, 34],
 24: [23, 35],
 25: [24, 36],
 26: [25, 37],
 27: [26, 38],
 28: [27, 39],
 29: [28, 40],
 30: [29, 41],
 31: [30, 42],
 32: [31, 43],
 33: [44],
 34: [33, 45],
 35: [34, 46],
 36: [35, 47],
 37: [36, 48],
 38: [37, 49],
 39: [38, 50],
 40: [39, 51],
 41: [40, 52],
 42: [41, 53],
 43: [42, 54],
 44: [55],
 45: [44, 56],
 46: [45, 57],
 47: [46, 58],
 48: [47, 59],
 49: [48, 60],
 50: [49, 61],
 51: [50, 62],
 52: [51, 63],
 53: [52, 64],
 54: [53, 65],
 55: [],
 56: [55],
 57: [56],
 58: [57],
 59: [58],
 60: [59],
 61: [60],
 62: [61],
 63: [62],
 64: [63],
 65: [64],
 66: [55],
 67: [66, 56],
 68: [67, 57],
 69: [68, 58],
 70: [69, 59],
 71: [70, 60],
 72: [71, 

In [5]:
# For each scenario, identify which other scenarios depend on it and can therefore be executed afterwards.
next_scenarios = [[] for i in range(n_scenarios)]

for current_scenario, adjacent_scenarios in enumerate(load_scenario_optimizer_states):
    for adjacent_scenario in adjacent_scenarios:
        next_scenarios[adjacent_scenario].append(current_scenario)

{i: n for i, n in enumerate(next_scenarios)}

{0: [1],
 1: [2],
 2: [3],
 3: [4],
 4: [5],
 5: [6],
 6: [7],
 7: [8],
 8: [9],
 9: [10],
 10: [],
 11: [0, 12],
 12: [1, 13],
 13: [2, 14],
 14: [3, 15],
 15: [4, 16],
 16: [5, 17],
 17: [6, 18],
 18: [7, 19],
 19: [8, 20],
 20: [9, 21],
 21: [10],
 22: [11, 23],
 23: [12, 24],
 24: [13, 25],
 25: [14, 26],
 26: [15, 27],
 27: [16, 28],
 28: [17, 29],
 29: [18, 30],
 30: [19, 31],
 31: [20, 32],
 32: [21],
 33: [22, 34],
 34: [23, 35],
 35: [24, 36],
 36: [25, 37],
 37: [26, 38],
 38: [27, 39],
 39: [28, 40],
 40: [29, 41],
 41: [30, 42],
 42: [31, 43],
 43: [32],
 44: [33, 45],
 45: [34, 46],
 46: [35, 47],
 47: [36, 48],
 48: [37, 49],
 49: [38, 50],
 50: [39, 51],
 51: [40, 52],
 52: [41, 53],
 53: [42, 54],
 54: [43],
 55: [44, 56, 66],
 56: [45, 57, 67],
 57: [46, 58, 68],
 58: [47, 59, 69],
 59: [48, 60, 70],
 60: [49, 61, 71],
 61: [50, 62, 72],
 62: [51, 63, 73],
 63: [52, 64, 74],
 64: [53, 65, 75],
 65: [54, 76],
 66: [67, 77],
 67: [68, 78],
 68: [69, 79],
 69: [70, 80],
 

In [6]:
# Establish the processing sequence such that all dependencies are fulfilled at each step, i.e., for each scenario once it is run
sequence = [55]

def append_next_to_sequence_after(current_scenario, sequence):
    # Check the scenarios that depend on this scenario
    for next_scenario in next_scenarios[current_scenario]:
        # Only consider adding the following scenario if it's not already in the sequence
        if next_scenario not in sequence:
            # Make sure not only the current scenario, but also all other scenarios the following scenario may rely on are already in the sequence before adding the following scenario. Otherwise, do nothing, and it will be added later on.
            if all([adjacent_scenario in sequence for adjacent_scenario in load_scenario_optimizer_states[next_scenario]]):
                sequence.append(next_scenario)
                # Follow the cascade: Check whether to add the scenarios following the scenario just added.
                append_next_to_sequence_after(next_scenario, sequence)

append_next_to_sequence_after(55, sequence)

order_correct = []

for i, s in enumerate(sequence):
    all_dependencies_fulfilled = [scenario in sequence[:i] for scenario in load_scenario_optimizer_states[s]]
    print(f"{s}: {load_scenario_optimizer_states[s]} {all_dependencies_fulfilled}")
    order_correct.extend(all_dependencies_fulfilled)
print(len(sequence))
print(all(order_correct))

55: [] []
44: [55] [True]
33: [44] [True]
22: [33] [True]
11: [22] [True]
0: [11] [True]
56: [55] [True]
45: [44, 56] [True, True]
34: [33, 45] [True, True]
23: [22, 34] [True, True]
12: [11, 23] [True, True]
1: [0, 12] [True, True]
57: [56] [True]
46: [45, 57] [True, True]
35: [34, 46] [True, True]
24: [23, 35] [True, True]
13: [12, 24] [True, True]
2: [1, 13] [True, True]
58: [57] [True]
47: [46, 58] [True, True]
36: [35, 47] [True, True]
25: [24, 36] [True, True]
14: [13, 25] [True, True]
3: [2, 14] [True, True]
59: [58] [True]
48: [47, 59] [True, True]
37: [36, 48] [True, True]
26: [25, 37] [True, True]
15: [14, 26] [True, True]
4: [3, 15] [True, True]
60: [59] [True]
49: [48, 60] [True, True]
38: [37, 49] [True, True]
27: [26, 38] [True, True]
16: [15, 27] [True, True]
5: [4, 16] [True, True]
61: [60] [True]
50: [49, 61] [True, True]
39: [38, 50] [True, True]
28: [27, 39] [True, True]
17: [16, 28] [True, True]
6: [5, 17] [True, True]
62: [61] [True]
51: [50, 62] [True, True]
40: [

The whole thing baked into a function:

In [1]:
from experiment.utils import get_processing_sequence

In [7]:
load_adjacent, next_scen, sequence_2 = get_processing_sequence()

In [8]:
sequence == sequence_2

True

In [7]:
print(", ".join([str(i) for i in sequence]))

55, 44, 33, 22, 11, 0, 56, 45, 34, 23, 12, 1, 57, 46, 35, 24, 13, 2, 58, 47, 36, 25, 14, 3, 59, 48, 37, 26, 15, 4, 60, 49, 38, 27, 16, 5, 61, 50, 39, 28, 17, 6, 62, 51, 40, 29, 18, 7, 63, 52, 41, 30, 19, 8, 64, 53, 42, 31, 20, 9, 65, 54, 43, 32, 21, 10, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142


## Sequential optimization test

In [5]:
e.optimize_reconstruction_params(sequence=[56], init_points=0, n_iter=50)

Optimization of reconstruction parameters for scenario scenario_056.
- Finding optimization logs of scenario: scenario_055
- scenario_055: optimization_240919-120720.log

Setting up reconstruction optimization ...
- Setting up experiment ...
- Setting up optimizer ...

Loading optimizer state from log file optimization_240919-120720.log ...
- Optimizer is now aware of 201 observations.
- Of these, 201 are actual target values.

Preparing reconstruction optimization ...
Saving experiment configuration ...
Saving scenario configurations ...
- Setting up logger ...

Running reconstruction optimization ...

Starting optimization scenario 'optim_0000' with the following settings:
{
  "r_line_epsilon": 1.0,
  "r_optimisation_data_term": 10.0,
  "r_plane_epsilon": 2.0,
  "r_plane_min_points": 13,
  "thres_alpha": 1.0,
  "thres_reg_line_dist": 1.0,
  "thres_reg_line_ext": 3.0,
  "r_plane_k": 13,
  "scenario_name": "optim_0000"
}

Starting 3D building reconstruction ...

- Command: geof C:/User

KeyError: 'iou_22_mean'

In [5]:
e[56].setup_reconstruction_optimization()


Setting up reconstruction optimization ...
- Setting up experiment ...
- Setting up optimizer ...


In [6]:
e[56].recon_optim.load_optim_experiment()

Loading experiment configuration ...
File `scenario_settings.json` not found.
Initializing experiment ...
Loading scenarios ...


In [7]:
e[56].recon_optim.optim_experiment.__len__()

50

In [8]:
e[56].recon_optim.select_optimal_scenario_multirank()


Selecting optimal scenario from all optimization scenarios.
Running additional evaluators for the optimization experiment ..

Running 'setup_evaluation' for optim_0000 ...

Finished 'setup_evaluation' for optim_0000 after 0:00:00.020036.

Running 'setup_evaluation' for optim_0001 ...

Finished 'setup_evaluation' for optim_0001 after 0:00:00.016007.

Running 'setup_evaluation' for optim_0002 ...

Finished 'setup_evaluation' for optim_0002 after 0:00:00.015491.

Running 'setup_evaluation' for optim_0003 ...

Finished 'setup_evaluation' for optim_0003 after 0:00:00.012991.

Running 'setup_evaluation' for optim_0004 ...

Finished 'setup_evaluation' for optim_0004 after 0:00:00.014001.

Running 'setup_evaluation' for optim_0005 ...

Finished 'setup_evaluation' for optim_0005 after 0:00:00.016033.

Running 'setup_evaluation' for optim_0006 ...

Finished 'setup_evaluation' for optim_0006 after 0:00:00.016995.

Running 'setup_evaluation' for optim_0007 ...

Finished 'setup_evaluation' for opt

In [10]:
e[56].recon_optim.optimal_scenario.name

'optim_0024'

## Single scenario optimization

In [5]:
e[0].setup_reconstruction_optimization()


Setting up reconstruction optimization ...
- Setting up experiment ...
- Setting up optimizer ...


In [6]:
e[0].recon_optim.load_optimizer_state(
    [r"C:\Users\Florian\Data\city-to-scan-to-city\utrecht_10-492-594\06_reconstruction_optimization\scenario_055\optimization_240919-120720.log"],
    update_iter_count=False
)


Loading optimizer state from log file optimization_240919-120720.log ...
- Optimizer is now aware of 201 observations.
- Of these, 201 are actual target values.


In [7]:
e[0].prepare_reconstruction_optimization()


Preparing reconstruction optimization ...
Saving experiment configuration ...
Saving scenario configurations ...
- Setting up logger ...


In [8]:
o = e[0].recon_optim.optimizer
len(o.res), len(o.space), len(o._queue), o._queue.empty

(201, 201, 0, True)

In [10]:
o._space.random_sample()

array([ 0.87955853,  8.00557506,  1.4453379 , 10.41168989,  0.97291887,
        0.84919838,  1.42467822])

In [9]:
e[0].run_reconstruction_optimization(init_points=3, n_iter=4)


Running reconstruction optimization ...

Starting optimization scenario 'optim_0000' with the following settings:
{
  "r_line_epsilon": 0.6391837420324455,
  "r_optimisation_data_term": 9.84952263408362,
  "r_plane_epsilon": 1.017570370645304,
  "r_plane_min_points": 27,
  "thres_alpha": 0.31086382277260965,
  "thres_reg_line_dist": 0.8893617297362744,
  "thres_reg_line_ext": 2.5476763525626707,
  "r_plane_k": 27,
  "scenario_name": "optim_0000"
}

Starting 3D building reconstruction ...

- Command: geof C:/Users/Florian/Data/city-to-scan-to-city/utrecht_10-492-594/06_reconstruction_optimization/scenario_000/07_reconstruction/optim_0000/reconstruct.json --verbose --workdir --config C:/Users/Florian/Data/city-to-scan-to-city/utrecht_10-492-594/06_reconstruction_optimization/scenario_000/07_reconstruction/optim_0000/config.toml
- Output log file: C:\Users\Florian\Data\city-to-scan-to-city\utrecht_10-492-594\06_reconstruction_optimization\scenario_000\07_reconstruction\optim_0000\geoflow

### Test duration of IOU evaluator

In [5]:
scenario_to_optimize_idx = 0
# Additionally compute 3D IOU as evaluation metric for the reconstruction optimization
eo = Experiment.load(e[scenario_to_optimize_idx].recon_optim_output_dirpath, load_scenarios=True)
eo.run_steps(Scenario.setup_evaluation, lods=["2.2"])
# eo.run_steps(Scenario.run_evaluation, evaluator_selection=["iou_3d"])

Loading experiment configuration ...
File `scenario_settings.json` not found.
Initializing experiment ...
Loading scenarios ...

Running 'setup_evaluation' for optim_0000 ...

Finished 'setup_evaluation' for optim_0000 after 0:00:00.014516.

Running 'setup_evaluation' for optim_0001 ...

Finished 'setup_evaluation' for optim_0001 after 0:00:00.014001.

Running 'setup_evaluation' for optim_0002 ...

Finished 'setup_evaluation' for optim_0002 after 0:00:00.013998.

Running 'setup_evaluation' for optim_0003 ...

Finished 'setup_evaluation' for optim_0003 after 0:00:00.012004.

Running 'setup_evaluation' for optim_0004 ...

Finished 'setup_evaluation' for optim_0004 after 0:00:00.013994.

Running 'setup_evaluation' for optim_0005 ...

Finished 'setup_evaluation' for optim_0005 after 0:00:00.019006.

Running 'setup_evaluation' for optim_0006 ...

Finished 'setup_evaluation' for optim_0006 after 0:00:00.014009.

Finished running 1 steps for 7 scenarios after 0:00:00.101529.


In [7]:
eo.compute_summary_statistics(evaluator_selection="iou_3d")

Computing summary statistics from all scenarios ...

